# Speaker Identification

In [1]:
import torch
import matplotlib.pyplot as plt
import torchaudio
import numpy as np
import time
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
import gc
import torch.nn.functional as F
import ot
import torch.nn as nn
import time
import util

In [2]:
class CustomDataset(Dataset):
    def __init__(self, idx, clean_data, noise_data):
        self.idx = idx
        self.clean_data = clean_data
        self.noise_data = noise_data

    def __len__(self):
        return len(self.clean_data)

    def __getitem__(self, index):
        idx = int(self.idx[index])
        clean_data = self.clean_data[index]
        noise_data = self.noise_data[index]
        return idx, clean_data, noise_data

In [3]:
TRAIN_DATASET = torch.load('data/DATASET_MFCC/TRAIN_SPEAKER_CN.pt')
VAL_DATASET_C = torch.load('data/DATASET_MFCC/VALIDATION_SPEAKER_C.pt')
TEST_DATASET_C = torch.load('data/DATASET_MFCC/TEST_SPEAKER_C.pt')
VAL_DATASET_D = torch.load('data/DATASET_MFCC/VALIDATION_SPEAKER_D.pt')
TEST_DATASET_D = torch.load('data/DATASET_MFCC/TEST_SPEAKER_D.pt')

In [4]:
BATCH_SIZE = 8
IS_SUFFLE = True
torch.manual_seed(0)
train_dataloader = DataLoader(TRAIN_DATASET, batch_size=BATCH_SIZE, shuffle=IS_SUFFLE)
val_dataloader_c = DataLoader(VAL_DATASET_C, batch_size=BATCH_SIZE, shuffle=IS_SUFFLE)
test_dataloader_c = DataLoader(TEST_DATASET_C, batch_size=BATCH_SIZE, shuffle=IS_SUFFLE)
val_dataloader_d = DataLoader(VAL_DATASET_D, batch_size=BATCH_SIZE, shuffle=IS_SUFFLE)
test_dataloader_d = DataLoader(TEST_DATASET_D, batch_size=BATCH_SIZE, shuffle=IS_SUFFLE)

In [5]:
for i, (idx, clean_data, noise_data) in enumerate(train_dataloader):
    print(i)
    print(idx)

0
tensor([3, 7, 8, 4, 7, 6, 5, 9])
1
tensor([7, 5, 5, 5, 0, 5, 3, 7])
2
tensor([1, 5, 6, 1, 2, 0, 3, 8])
3
tensor([0, 1, 7, 3, 0, 8, 0, 9])
4
tensor([0, 4, 3, 3, 6, 5, 2, 0])
5
tensor([0, 5, 3, 7, 2, 6, 5, 4])
6
tensor([7, 5, 7, 9, 1, 1, 4, 5])
7
tensor([7, 5, 9, 3, 6, 1, 4, 4])
8
tensor([7, 4, 9, 7, 4, 9, 6, 4])
9
tensor([6, 7, 0, 7, 0, 2, 8, 6])
10
tensor([7, 7, 6, 0, 2, 3, 7, 5])
11
tensor([6, 7, 8, 6, 6, 5, 5, 7])
12
tensor([0, 5, 2, 4, 7, 3, 8, 0])
13
tensor([7, 9, 0, 8, 0, 3, 1, 9])
14
tensor([5, 6, 5, 5, 5, 5, 3, 0])
15
tensor([7, 6, 6, 4, 0, 7, 1, 7])
16
tensor([1, 1, 5, 6, 4, 0, 3, 0])
17
tensor([5, 7, 5, 1, 3, 4, 3, 1])
18
tensor([9, 6, 7, 8, 9, 7, 1, 2])
19
tensor([3, 2, 7, 3, 3, 7, 3, 5])
20
tensor([7, 8, 8, 8, 0, 5, 5, 6])
21
tensor([4, 3, 2, 6, 0, 6, 7, 6])
22
tensor([1, 5, 6, 6, 6, 8, 1, 6])
23
tensor([8, 0, 2, 3, 9, 6, 3, 8])
24
tensor([1, 0, 5, 0, 2, 4, 3, 4])
25
tensor([5, 1, 7, 2, 5, 4, 9, 7])
26
tensor([0, 0, 2, 3, 6, 1, 4, 6])
27
tensor([1, 7, 6, 1, 7, 7, 4, 3])
28

# Create model

In [6]:
class VGG16(nn.Module):
    def __init__(self, num_classes=100):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True))
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True), 
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding=0, dilation=1, ceil_mode=False))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True))
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding=0, dilation=1, ceil_mode=False))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True))
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True))
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding=0, dilation=1, ceil_mode=False))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True))
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True))
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding=0, dilation=1, ceil_mode=False))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True))
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True))
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding=0, dilation=1, ceil_mode=False))
        self.fc = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(8192, 4096, bias=True),
            nn.ReLU(inplace=True))
        self.fc1 = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(4096, 4096, bias=True),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes, bias=True))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        #print(out.size())
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [7]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.blstm = nn.LSTM(40, 512, dropout=0.0, num_layers=2, bidirectional=True, batch_first=True)
        self.LReLU = nn.LeakyReLU(0.3)
        self.ReLU = nn.ReLU()
        self.Dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(512 * 2, 512)
        self.fc2 = nn.Linear(512, 40)

    def forward(self, x):
        #  x: clean mag, y: noise mag
        output, _ = self.blstm(x)
        output = self.fc1(output)
        output = self.LReLU(output)
        output = self.Dropout(output)
        output = self.fc2(output)
        output = self.ReLU(output)
        return output

In [8]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.conv1 = nn.Sequential(  # input shape (batch_size, 1, 64, 257)
            nn.Conv2d(
                in_channels=1,  # input height
                out_channels=8,  # n_filters
                kernel_size=5,  # filter size
                stride=1,  # filter movement/step
                padding=2,
                # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
            ),  # output shape (16, 28, 28)
            nn.ReLU(),  # activation
            nn.MaxPool2d(kernel_size=2),  # choose max value in 2x2 area, output shape (batch_size, 16, 32, 128)
        )
        self.conv2 = nn.Sequential(  # input shape (batch_size, 16, 32, 128)
            nn.Conv2d(8, 16, 5, 1, 2),
            nn.ReLU(),  # activation
            nn.MaxPool2d(2),  # output shape (batch_size, 32, 16, 64)
        )
        self.out1 = nn.Sequential(
            nn.Linear(20480, 16 * 16),  # fully connected layer, output 10 classes
            nn.ReLU()  # activation
        )

        self.out2 = nn.Sequential(
            nn.Linear(16 * 16, 1),  # fully connected layer, output 10 classes
            nn.Sigmoid()  # activation
        )

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)  # flatten the output of conv2 to (batch_size, 32 * 16 * 64)
        x = self.out1(x)
        output = self.out2(x)
        return output

In [9]:
def OT_loss(X_s, X_t, y_s, y_t_pred):
    N = X_s.shape[0]

    C0 = torch.cdist(X_s.reshape((N, -1)), X_t.reshape((N, -1)), p=2).cpu()
    C1 = torch.cdist(y_s.reshape((N, -1)), y_t_pred.reshape((N, -1)), p=2).cpu()

    alpha = 1  # OT source weight in loss
    beta = 1   # OT target weight in loss
    C = alpha * C0 + beta * C1

    γ = ot.emd(ot.unif(N), ot.unif(N), C.detach().numpy())
    γ = torch.from_numpy(γ).float()

    loss = torch.sum(γ * C)

    return loss

In [10]:
"""
def OT_loss1(y_s, y_t_pred):
    N = y_s.shape[0]

    #C0 = torch.cdist(X_s.reshape((N, -1)), X_t.reshape((N, -1)), p=2).cpu()
    C1 = torch.cdist(y_s.reshape((N, -1)), y_t_pred.reshape((N, -1)), p=2).cpu()

    #alpha = 1  # OT source weight in loss
    beta = 1   # OT target weight in loss
    C = beta * C1

    γ = ot.emd(ot.unif(N), ot.unif(N), C.detach().numpy())
    γ = torch.from_numpy(γ).float()

    loss = torch.sum(γ * C)

    return loss"""

'\ndef OT_loss1(y_s, y_t_pred):\n    N = y_s.shape[0]\n\n    #C0 = torch.cdist(X_s.reshape((N, -1)), X_t.reshape((N, -1)), p=2).cpu()\n    C1 = torch.cdist(y_s.reshape((N, -1)), y_t_pred.reshape((N, -1)), p=2).cpu()\n\n    #alpha = 1  # OT source weight in loss\n    beta = 1   # OT target weight in loss\n    C = beta * C1\n\n    γ = ot.emd(ot.unif(N), ot.unif(N), C.detach().numpy())\n    γ = torch.from_numpy(γ).float()\n\n    loss = torch.sum(γ * C)\n\n    return loss'

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device available: {0}'.format(device))

Device available: cuda:0


In [12]:
C = VGG16(num_classes=10).to(device)
G = Generator().to(device)
D = Discriminator().to(device)

In [13]:
for i, (idx, clean_data, noise_data) in enumerate(train_dataloader):
    print(clean_data.unsqueeze(1).size())
    #output = model(clean_data.unsqueeze(1).to(device))
    break

torch.Size([8, 1, 512, 40])


In [14]:
hyperparameters = {
    'NUM_EPOCHS' : 150,
    'optimizer_g' : torch.optim.Adam(G.parameters(), lr=1e-5),
    'optimizer_d' : torch.optim.Adam(D.parameters(), lr=1e-3),
    'optimizer_c' : torch.optim.Adam(C.parameters(), lr=1e-4),
    'optimizer_ot' : torch.optim.Adam(C.parameters(), lr=1e-5),
    'criterion_d' : nn.BCELoss(),
    'criterion_c' : nn.CrossEntropyLoss()
}

In [15]:
LOSS_TRAIN_DATALOADER = []
LOSS_VAL_DATALOADER_C = []
LOSS_VAL_DATALOADER_D = []
LOSS_VAL_DATALOADER_N = []

ACC_TRAIN_DATALOADER = []
ACC_VAL_DATALOADER_C = []
ACC_VAL_DATALOADER_D = []
ACC_VAL_DATALOADER_N = []

TIMES_PER_EPOCH = []
for epoch in range(hyperparameters['NUM_EPOCHS']):
    running_loss = 0.0
    start_time = time.time()
    #model.train()
    for idx, (labels, clean_data, noise_data) in enumerate(train_dataloader):
        labels = labels.to(device)
        clean_data = clean_data.to(device)
        #denoised_data = denoised_data.to(device)
        noise_data = noise_data.to(device)
        
        tmpBatchSize = len(labels)
        
        true_label = torch.ones(tmpBatchSize, 1, device=device)
        fake_label = torch.zeros(tmpBatchSize, 1, device=device)

        fakeImageBatch = G(noise_data)
        
        # train discriminator on real images
        predictionsReal = D(clean_data)
        lossDiscriminator = hyperparameters['criterion_d'](predictionsReal, true_label) #labels = 1
        lossDiscriminator.backward(retain_graph = True)
        
        # train discriminator on fake images
        predictionsFake = D(fakeImageBatch)
        lossFake = hyperparameters['criterion_d'](predictionsFake, fake_label) #labels = 0
        lossFake.backward(retain_graph= True)
        hyperparameters['optimizer_d'].step() # update discriminator parameters
        
        # train generator 
        hyperparameters['optimizer_g'].zero_grad()
        predictionsFake = D(fakeImageBatch)
        lossGenerator = hyperparameters['criterion_d'](predictionsFake, true_label) #labels = 1
        lossGenerator.backward(retain_graph = True)
        hyperparameters['optimizer_g'].step()
        
        torch.autograd.set_detect_anomaly(True)
        fakeImageBatch = fakeImageBatch.detach().clone()
        
        #if (idx % 2 == 0):
        #    # train classifier on real data
        #    predictions = C(denoised_data.unsqueeze(1))
        #    realClassifierLoss = OT_loss(predictions, F.one_hot(labels, num_classes=20).float())#criterion(predictions, labels)
        #    realClassifierLoss.backward(retain_graph=True)
        #    hyperparameters['optimizer_ot'].step()
        #    hyperparameters['optimizer_ot'].zero_grad()
        
        
            #loss_s = hyperparameters['criterion_c'](C(denoised_data.unsqueeze(1)), labels)
            #hyperparameters['optimizer_c'].zero_grad()
            #loss_s.backward()
            #hyperparameters['optimizer_c'].step()
        
        # train classifier on real data
        predictions = C(clean_data.unsqueeze(1))
        realClassifierLoss = OT_loss(clean_data, noise_data, predictions, F.one_hot(labels, num_classes=10).float())#criterion(predictions, labels)
        #realClassifierLoss = OT_loss(predictions, F.one_hot(labels, num_classes=10).float())#criterion(predictions, labels)

        realClassifierLoss.backward(retain_graph=True)
        
        hyperparameters['optimizer_ot'].step()
        hyperparameters['optimizer_ot'].zero_grad()
        
        # update the classifer on fake data
        predictionsFake = C(fakeImageBatch.unsqueeze(1))
        predictedLabels = torch.argmax(predictionsFake, 1) # -> [0 , 5, 9, 3, ...]
        confidenceThresh = .7 # for example
        probs = F.softmax(predictionsFake, dim=1)
        mostLikelyProbs = np.asarray([probs[i, predictedLabels[i]].item() for  i in range(len(probs))])
        toKeep = mostLikelyProbs > confidenceThresh
        if sum(toKeep) != 0:
            fakeClassifierLoss = hyperparameters['criterion_c'](predictionsFake[toKeep], predictedLabels[toKeep])
            fakeClassifierLoss.backward()
        hyperparameters['optimizer_c'].step()
        
        
        
        # reset the gradients
        hyperparameters['optimizer_d'].zero_grad()
        hyperparameters['optimizer_g'].zero_grad()
        hyperparameters['optimizer_ot'].zero_grad()
        
        del labels, clean_data, noise_data, true_label, fake_label
        torch.cuda.empty_cache()
        gc.collect()
        
        
    
    
    
    
    
    
    
    


    # Scores for train and validation
    epoch_time = time.time() - start_time
    #TIME.append(epoch_time)
    print(f"Epoch {epoch+1} took {epoch_time:.2f} seconds\n========================================================")    
    TIMES_PER_EPOCH.append(epoch_time)
    with torch.no_grad():
        run_loss_train = 0.0
        correct = 0
        total = 0
        for i, (labels, clean_data, _) in enumerate(train_dataloader):
            labels = labels.to(device)
            clean_data = clean_data.unsqueeze(1).to(device)
            outputs = C(clean_data)
            loss_train = hyperparameters['criterion_c'](outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            run_loss_train += loss_train.item()
            del labels, clean_data, outputs
        print('TRAIN - Loss of the network on the {} training MFCCs: {}'.format(len(train_dataloader.dataset), run_loss_train/len(train_dataloader))) 
        print('TRAIN - Accuracy of the network on the {} training MFCCs: {} %'.format(len(train_dataloader.dataset), 100 * correct / total))
        LOSS_TRAIN_DATALOADER.append(run_loss_train/len(train_dataloader))
        ACC_TRAIN_DATALOADER.append(100 * correct / total)
        
    with torch.no_grad():
        run_loss_train = 0.0
        correct = 0
        total = 0
        for i, (labels, clean_data, _) in enumerate(val_dataloader_c):
            labels = labels.to(device)
            clean_data = clean_data.unsqueeze(1).to(device)
            outputs = C(clean_data)
            loss_train = hyperparameters['criterion_c'](outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            run_loss_train += loss_train.item()
            del labels, clean_data, outputs
        print('VAL-CLEAN - Loss of the network on the {} training MFCCs: {}'.format(len(val_dataloader_c.dataset), run_loss_train/len(val_dataloader_c))) 
        print('VAL-CLEAN - Accuracy of the network on the {} training MFCCs: {} %'.format(len(val_dataloader_c.dataset), 100 * correct / total))
        LOSS_VAL_DATALOADER_C.append(run_loss_train/len(val_dataloader_c))
        ACC_VAL_DATALOADER_C.append(100 * correct / total)
    with torch.no_grad():
        run_loss_train = 0.0
        correct = 0
        total = 0
        for i, (labels, clean_data, _) in enumerate(val_dataloader_d):
            labels = labels.to(device)
            clean_data = clean_data.unsqueeze(1).to(device)
            outputs = C(clean_data)
            loss_train = hyperparameters['criterion_c'](outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            run_loss_train += loss_train.item()
            del labels, clean_data, outputs
        print('VAL-DENOISED - Loss of the network on the {} training MFCCs: {}'.format(len(val_dataloader_d.dataset), run_loss_train/len(val_dataloader_d))) 
        print('VAL-DENOISED - Accuracy of the network on the {} training MFCCs: {} %'.format(len(val_dataloader_d.dataset), 100 * correct / total))
        LOSS_VAL_DATALOADER_D.append(run_loss_train/len(val_dataloader_d))
        ACC_VAL_DATALOADER_D.append(100 * correct / total)
    with torch.no_grad():
        run_loss_train = 0.0
        correct = 0
        total = 0
        for i, (labels, _, clean_data) in enumerate(val_dataloader_d):
            labels = labels.to(device)
            clean_data = clean_data.unsqueeze(1).to(device)
            outputs = C(clean_data)
            loss_train = hyperparameters['criterion_c'](outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            run_loss_train += loss_train.item()
            del labels, clean_data, outputs
        print('VAL-NOISE - Loss of the network on the {} training MFCCs: {}'.format(len(val_dataloader_d.dataset), run_loss_train/len(val_dataloader_d))) 
        print('VAL-NOISE - Accuracy of the network on the {} training MFCCs: {} %'.format(len(val_dataloader_d.dataset), 100 * correct / total))
        LOSS_VAL_DATALOADER_N.append(run_loss_train/len(val_dataloader_d))
        ACC_VAL_DATALOADER_N.append(100 * correct / total)

Epoch 1 took 133.31 seconds
TRAIN - Loss of the network on the 3200 training MFCCs: 1.9968709552288055
TRAIN - Accuracy of the network on the 3200 training MFCCs: 60.625 %
VAL-CLEAN - Loss of the network on the 200 training MFCCs: 2.045601177215576
VAL-CLEAN - Accuracy of the network on the 200 training MFCCs: 50.5 %
VAL-DENOISED - Loss of the network on the 200 training MFCCs: 1.9828209686279297
VAL-DENOISED - Accuracy of the network on the 200 training MFCCs: 69.5 %
VAL-NOISE - Loss of the network on the 200 training MFCCs: 2.206805543899536
VAL-NOISE - Accuracy of the network on the 200 training MFCCs: 30.0 %
Epoch 2 took 140.88 seconds
TRAIN - Loss of the network on the 3200 training MFCCs: 1.821680255830288
TRAIN - Accuracy of the network on the 3200 training MFCCs: 77.96875 %
VAL-CLEAN - Loss of the network on the 200 training MFCCs: 1.8698299932479858
VAL-CLEAN - Accuracy of the network on the 200 training MFCCs: 69.5 %
VAL-DENOISED - Loss of the network on the 200 training MFCC

In [16]:
torch.save(C.state_dict(), 'models_classification/C_model_SPEAKER_IDENTIFICATION_FINAL.pt')
torch.save(G.state_dict(), 'models_classification/G_model_SPEAKER_IDENTIFICATION_FINAL.pt')
torch.save(D.state_dict(), 'models_classification/D_model_SPEAKER_IDENTIFICATION_FINAL.pt')

In [17]:
util.generate_pkl('stastics/', LOSS_TRAIN_DATALOADER, 'LOSS_TRAIN_DATALOADER')
util.generate_pkl('stastics/', LOSS_VAL_DATALOADER_C, 'LOSS_VAL_DATALOADER_C')
util.generate_pkl('stastics/', LOSS_VAL_DATALOADER_D, 'LOSS_VAL_DATALOADER_D')
util.generate_pkl('stastics/', LOSS_VAL_DATALOADER_N, 'LOSS_VAL_DATALOADER_N')
util.generate_pkl('stastics/', ACC_TRAIN_DATALOADER, 'ACC_TRAIN_DATALOADER')
util.generate_pkl('stastics/', ACC_VAL_DATALOADER_C, 'ACC_VAL_DATALOADER_C')
util.generate_pkl('stastics/', ACC_VAL_DATALOADER_D, 'ACC_VAL_DATALOADER_D')
util.generate_pkl('stastics/', ACC_VAL_DATALOADER_N, 'ACC_VAL_DATALOADER_N')
util.generate_pkl('stastics/', TIMES_PER_EPOCH, 'TIMES_PER_EPOCH')

[1.9968709552288055, 1.821680255830288, 1.7261099880933761, 1.6538476434350013, 1.5741830024123191, 1.558623428940773, 1.61464220225811, 1.544168854057789, 1.5376863291859626, 1.5236787593364716, 1.5439169776439667, 1.4989723578095435, 1.4553104057908057, 1.4691202232241631, 1.5000660008192062, 1.4796025922894478, 1.492739024758339, 1.4719398948550224, 1.501029182076454, 1.459610752761364, 1.4702171939611435, 1.4459092390537263, 1.4546281132102012, 1.4728789669275284, 1.4894529908895493, 1.4863577803969383, 1.4598145270347596, 1.4781362468004227, 1.4856143620610238, 1.4734770238399506, 1.4860741141438485, 1.4523989206552506, 1.444355897307396, 1.465583052635193, 1.472646006643772, 1.5060332152247429, 1.4546535530686378, 1.4831893903017044, 1.4780607563257218, 1.4765792512893676, 1.4744442144036294, 1.483465665578842, 1.478320088982582, 1.4524567124247552, 1.4659788709878923, 1.4735528826713562, 1.482878700196743, 1.460726135969162, 1.4507549387216567, 1.4720946550369263, 1.467289858162